## Examining the relationship between crude oil prices and gasoline prices

In [ ]:
import os
import requests
import pandas as pd
from pandas.tseries.offsets import DateOffset
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
import statsmodels.api as sm
import numpy as np
import pymc3 as pm
import json
import arviz as az


In [ ]:
eia_api_key = os.environ['EIA_API_KEY']

In [ ]:
wti_api_url = r'https://api.eia.gov/series/?api_key=' + eia_api_key + '&series_id=PET.RWTC.D'
response = requests.get(wti_api_url)
wti = response.json()
df_wti = pd.DataFrame(wti['series'][0]['data'])
df_wti.rename(columns={0: 'date', 1: 'wti'}, inplace=True)
df_wti['date'] = pd.to_datetime(df_wti.date)
df_wti.sort_values(by='date', inplace=True)
df_wti.tail()

In [ ]:
gasoline_api_url = r'https://api.eia.gov/series/?api_key=' + eia_api_key + '&series_id=PET.EMM_EPM0_PTE_NUS_DPG.W'
response = requests.get(gasoline_api_url)
gasoline = response.json()
df_gasoline = pd.DataFrame(gasoline['series'][0]['data'])
df_gasoline.rename(columns={0: 'date', 1: 'gasoline'}, inplace=True)
df_gasoline['date'] = pd.to_datetime(df_gasoline.date)
df_gasoline.sort_values(by='date', inplace=True)
df_gasoline.tail()

In [ ]:
def weekly_oil(df1, df2):
    wti_weekly = pd.DataFrame(columns=['date', 'wti'])
    wti_prices = []
    num_dates = len(df1.date.unique())
    for i, d in enumerate(df1.date.unique()):
        if i == 0:
            end = pd.Timestamp(d)
            start = end - DateOffset(days=7)
            wti_prices.append(df2.loc[(df2.date>start)&(df2.date<=end), 'wti'].mean())
        else:
            start = df1.date.iloc[i-1]
            end = d
            wti_prices.append(df2.loc[(df2.date>start)&(df2.date<=end), 'wti'].mean())
    wti_weekly['date'] = df1.date.unique()
    wti_weekly['wti'] = wti_prices
    return wti_weekly

df_wti_weekly = weekly_oil(df_gasoline, df_wti)

In [ ]:
df = df_gasoline.merge(df_wti_weekly, on='date')
df['year'] = df.date.dt.year

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,7))
ax2 = ax1.twinx()
g = sns.lineplot(data=df, x='date', y='gasoline', ax=ax1)
sns.lineplot(data=df, x='date', y='wti', ax=ax2, color='orange')
g.legend(handles=[Line2D([], [], color='blue', label='gasoline'),
                  Line2D([], [], color='orange', label='wti')], loc=(0.01,.9))
plt.savefig('wti and gasoline vs time.png')

In [ ]:
cross_corr = sm.tsa.stattools.ccf(df.wti, df.gasoline, adjusted=False)
fig, ax = plt.subplots(figsize=(15,7))
plt.bar(np.arange(1, len(df)+1), cross_corr, width=1)
plt.ylabel('correlation')
plt.xlabel('lag')
plt.savefig('wti vs gasoline cross correlation.png')

In [ ]:
df['gasoline_bbls'] = df.gasoline * 42
df['crack_spread'] = df.gasoline_bbls - df.wti
df.tail()

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,7))
sns.lineplot(data=df, x='date', y='wti')
sns.lineplot(data=df, x='date', y='gasoline_bbls')
plt.fill_between(df.date, df.wti, df.gasoline_bbls, alpha=0.33)

In [ ]:

fig, ax1 = plt.subplots(figsize=(15,7))
g = sns.lineplot(data=df, x='date', y='crack_spread', ax=ax1)
plt.ylabel('crack spread')
plt.savefig('crack spread vs time.png')

In [ ]:
headers = {'Content-type': 'application/json'}
data = json.dumps({'seriesid':  ['CUUR0000SA0E'],
                   'startyear': '1992', 
                   'endyear':   '2001'})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data, headers=headers)
cpi_data1 = json.loads(p.text)
print(cpi_data1['status'])
print(cpi_data1['message'])

headers = {'Content-type': 'application/json'}
data = json.dumps({'seriesid':  ['CUUR0000SA0E'],
                   'startyear': '2002', 
                   'endyear':   '2011'})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data, headers=headers)
cpi_data2 = json.loads(p.text)
print(cpi_data2['status'])
print(cpi_data2['message'])

headers = {'Content-type': 'application/json'}
data = json.dumps({'seriesid':  ['CUUR0000SA0E'],
                   'startyear': '2012', 
                   'endyear':   '2021'})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data, headers=headers)
cpi_data3 = json.loads(p.text)
print(cpi_data3['status'])
print(cpi_data3['message'])

headers = {'Content-type': 'application/json'}
data = json.dumps({'seriesid':  ['CUUR0000SA0E'],
                   'startyear': '2022', 
                   'endyear':   '2022'})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data, headers=headers)
cpi_data4 = json.loads(p.text)
print(cpi_data4['status'])
print(cpi_data4['message'])

In [ ]:
df_cpi = pd.DataFrame(columns=['year', 'period', 'value'])
year = []
period = []
value = []

for i in cpi_data1['Results']['series'][0]['data']:
    year.append(int(i['year']))
    if 'S' not in i['period']:
        period.append(int(i['period'].strip('M')))
    value.append(float(i['value']))

for i in cpi_data2['Results']['series'][0]['data']:
    year.append(int(i['year']))
    if 'S' not in i['period']:
        period.append(int(i['period'].strip('M')))
    value.append(float(i['value']))

for i in cpi_data3['Results']['series'][0]['data']:
    year.append(int(i['year']))
    if 'S' not in i['period']:
        period.append(int(i['period'].strip('M')))
    value.append(float(i['value']))

for i in cpi_data4['Results']['series'][0]['data']:
    year.append(int(i['year']))
    if 'S' not in i['period']:
        period.append(int(i['period'].strip('M')))
    value.append(float(i['value']))

df_cpi['year'] = year
df_cpi['period'] = period
df_cpi['value'] = value

In [ ]:
cpi_values = []
for i, row in df.iterrows():
    year = row['year']
    month = row['date'].month
    cpi = df_cpi.loc[(df_cpi.year == year) & (df_cpi.period == month), 'value'].values
    if len(cpi) > 0:
        cpi_values.append(cpi[0])
    else:
        cpi_values.append(cpi_values[i-1])
df['cpi_values'] = cpi_values
df['crack_spread_adj'] = df.crack_spread / df.cpi_values * 100
df['gasoline_adj'] = df.gasoline / df.cpi_values * 100
df['gasoline_bbls_adj'] = df.gasoline_bbls / df.cpi_values * 100
df['wti_adj'] = df.wti / df.cpi_values * 100

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,7))
g = sns.lineplot(data=df, x='date', y='cpi_values', ax=ax1)

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,7))
g = sns.lineplot(data=df, x='date', y='crack_spread_adj', ax=ax1)
plt.ylabel('crack spread inflation-adjusted')
plt.savefig('crack spread adjusted vs time.png')

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
sns.scatterplot(data=df, x='wti', y='gasoline')
plt.savefig('wti vs gasoline.png')

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
sns.scatterplot(data=df, x='wti', y='gasoline', hue='year')
plt.savefig('wti vs gasoline by year.png')

In [ ]:
sns.lmplot(data=df[df.year>=2008], x='wti', y='gasoline', hue='year', palette='rocket_r', height=7, aspect=1.667)
plt.savefig('wti vs gasoline by year fitted.png')

In [ ]:
sns.lmplot(data=df[df.year>=2008], x='wti_adj', y='gasoline_adj', hue='year', palette='rocket_r', height=7, aspect=1.667)
plt.savefig('wti vs gasoline adjusted by year fitted.png')

In [ ]:
sns.lmplot(data=df[df.year>=2008], x='wti', y='gasoline_bbls', height=7, aspect=1.667)

In [ ]:
sns.lmplot(data=df[df.year>=2008], x='wti_adj', y='gasoline_bbls_adj', height=7, aspect=1.667)

In [ ]:
X = df[df.year>=2008].wti_adj
y = df[df.year>=2008].gasoline_bbls_adj

with pm.Model() as model:
    b1 = pm.Normal('slope', 0, 100)
    b0 = pm.Normal('intercept', 0, 100)
    s = pm.Exponential('error', 1)
    x_ = pm.Data('features', X)

    obs = pm.Normal('observation', b1*x_ + b0, s, observed=y)
    
    trace = pm.sample(tune=10000)
    
az.plot_posterior(trace)
az.plot_trace(trace)

In [ ]:
x_new = np.linspace(0,60,61)
with model:
    pm.set_data({'features': x_new})
    posterior = pm.sample_posterior_predictive(trace)

y_pred = posterior['observation']
y_mean = y_pred.mean(axis=0)
y_std = y_pred.std(axis=0)

fig, ax = plt.subplots(figsize=(15,7))
sns.scatterplot(data=df[df.year>=2008], x='wti_adj', y='gasoline_bbls_adj')
plt.plot(x_new, y_mean, label='Prediction Mean')
plt.fill_between(x_new, y_mean - 3*y_std, y_mean + 3*y_std, alpha=0.33, label='Uncertainty Interval ($\mu\pm3\sigma$)')
plt.legend(loc='upper left')
plt.savefig('predicted wti vs gasoline.png')

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
sns.scatterplot(data=df, x='wti_adj', y='crack_spread_adj')

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
sns.scatterplot(data=df[df.year>=2008], x='wti_adj', y='crack_spread_adj', hue='year', palette='rocket_r')